In [5]:
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from scipy.integrate import solve_ivp
import torch
from torch import nn

In [2]:
animation.writer = animation.writers['ffmpeg']

## Generate data and animation from scratch

In [14]:
# Parameters
g = 9.81  # acceleration due to gravity (m/s^2)
l1 = 1.0  # length of the first rod (m)
l2 = 1.0  # length of the second rod (m)
m1 = 1.0  # mass of the first bob (kg)
m2 = 1.0  # mass of the second bob (kg)
t_final = 20
exp_name = 'start_almost_vertical'
num_tsteps = 2000

# Initial conditions
theta1_init = np.pi
theta2_init = np.pi + 1e-3
z1_init = 0.0            # initial angular velocity of the first pendulum (rad/s)
z2_init = 0.0            # initial angular velocity of the second pendulum (rad/s)

# Equations of motion
def derivatives(t, state):
    theta1, z1, theta2, z2 = state
    delta = theta2 - theta1

    denominator1 = (m1 + m2) * l1 - m2 * l1 * np.cos(delta) ** 2
    denominator2 = (l2 / l1) * denominator1

    dtheta1_dt = z1
    dz1_dt = (
        (m2 * l1 * z1 ** 2 * np.sin(delta) * np.cos(delta)
         + m2 * g * np.sin(theta2) * np.cos(delta)
         + m2 * l2 * z2 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta1))
        / denominator1
    )
    dtheta2_dt = z2
    dz2_dt = (
        (-m2 * l2 * z2 ** 2 * np.sin(delta) * np.cos(delta)
         + (m1 + m2) * g * np.sin(theta1) * np.cos(delta)
         - (m1 + m2) * l1 * z1 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta2))
        / denominator2
    )

    return np.array([dtheta1_dt, dz1_dt, dtheta2_dt, dz2_dt])


state_0 = np.array([theta1_init, z1_init, theta2_init, z2_init])
t = np.linspace(0, t_final, num_tsteps)  # time array

# Solve the system
solution = solve_ivp(derivatives, (0, t_final), state_0, t_eval=t)
theta1, theta2 = solution.y[0], solution.y[2]

# Convert to Cartesian coordinates
x1 = l1 * np.sin(theta1)
y1 = -l1 * np.cos(theta1)
x2 = x1 + l2 * np.sin(theta2)
y2 = y1 - l2 * np.cos(theta2)

# Animation
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-(l1+l2)*1.1, (l1+l2)*1.1)
ax.set_ylim(-(l1+l2)*1.1, (l1+l2)*1.1)
ax.set_aspect('equal')
line, = ax.plot([], [], 'o-', lw=2, color='blue')

def update(frame):
    line.set_data([0, x1[frame], x2[frame]], [0, y1[frame], y2[frame]])
    return line,

ani = FuncAnimation(fig, update, frames=len(t), interval=(t_final / 2), blit=True)
ani.save(f'../animations/{exp_name}.mp4')
plt.close(fig)

<video src='../animations/start_almost_vertical.mp4' controls>

## Pull data from generated dataset and animate

In [11]:
# Set animation parameters
row_of_dataset = 500
dataset_name = 'simulations-zero_initial_velocity-equal_mass-equal_length'

# Load and parse data
with open(f'../data/{dataset_name}.json', 'r') as f:
    data = json.load(f)
solution = data[row_of_dataset]
theta1, theta2 = solution['theta1'], solution['theta2']
num_tsteps, t_final = solution['num_tsteps'], solution['t_final']
l1, l2, m1, m2 = solution['length1'], solution['length2'], solution['mass1'], solution['mass2']

# Convert to Cartesian coordinates
x1 = l1 * np.sin(theta1)
y1 = -l1 * np.cos(theta1)
x2 = x1 + l2 * np.sin(theta2)
y2 = y1 - l2 * np.cos(theta2)

# Animation
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-(l1+l2)*1.1, (l1+l2)*1.1)
ax.set_ylim(-(l1+l2)*1.1, (l1+l2)*1.1)
ax.set_aspect('equal')
line, = ax.plot([], [], 'o-', lw=2, color='blue')

def update(frame):
    line.set_data([0, x1[frame], x2[frame]], [0, y1[frame], y2[frame]])
    return line,

ani = FuncAnimation(fig, update, frames=len(t), interval=(t_final / num_tsteps * 1000), blit=True)
ani.save(f'../animations/{dataset_name}.mp4')
plt.close(fig)

<video src='../animations/test_access.mp4' controls>

## Generate data from pretrained model and animate

In [6]:
class FeedForward(nn.Module):
    #So bog basic it should tap for black mana
    #DT That one's for you
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(8, 30)
        self.linear2 = nn.Linear(30,30)
        self.linear3 = nn.Linear(30,30)
        self.linear4 = nn.Linear(30,8)
        self.relu = nn.ReLU()
    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.relu(self.linear3(x))
        x = self.linear4(x)
        x = torch.clamp(x, -1e6, 1e6)
        return x

In [28]:
# Set parameters
model_name = 'after_240_epochs'
l1, l2 = 1, 1
m1, m2 = 1, 1
theta1_init, theta2_init = float(np.random.randn()), float(np.random.randn())
z1_init, z2_init = 0, 0
timestep = 10/1999 # don't change unless the dataset changes
total_time = 10

# Calculate other parameters
num_tsteps = int(total_time / timestep)

# Load model
model = FeedForward()
model = torch.load(f'../models/{model_name}.torch', weights_only=False)
model.eval()

# Generate data using model
init_state = torch.tensor([l1, l2, m1, m2, theta1_init, z1_init, theta2_init, z2_init])
solution = [init_state]
with torch.no_grad():
    for _ in range(int(total_time/timestep)):
        solution.append(model(solution[-1]))

theta1 = [sol[4] for sol in solution]
theta2 = [sol[6] for sol in solution]

# Convert to Cartesian coordinates
x1 = l1 * np.sin(theta1)
y1 = -l1 * np.cos(theta1)
x2 = x1 + l2 * np.sin(theta2)
y2 = y1 - l2 * np.cos(theta2)

# Animation
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_xlim(-(l1+l2)*1.1, (l1+l2)*1.1)
ax.set_ylim(-(l1+l2)*1.1, (l1+l2)*1.1)
ax.set_aspect('equal')
line, = ax.plot([], [], 'o-', lw=2, color='blue')

def update(frame):
    line.set_data([0, x1[frame], x2[frame]], [0, y1[frame], y2[frame]])
    return line,

ani = FuncAnimation(fig, update, frames=num_tsteps+1, interval=(total_time / num_tsteps * 1000), blit=True)
ani.save(f'../animations/{model_name}.mp4')
plt.close(fig)

<video src='../animations/after_240_epochs.mp4' controls>